<font size=3>**# 3.1 Object localization**</font>

**Classification with Localization vs. Detection**

The classification and the classification of localization: usually one obj. Detection: there can be multiple objs in an image.

**Output Bounding a Box to Localize**

Bisides features such as pedestrian, car, motorcycle and background, add a few output numbers: bx, by, bh, bw. And the training set should contain four numbers giving the bounding box.

 **Notations**

upper left: (0, 0), lower right: (1, 1)  
(bx, by): the midpoint

**Defining the Target Label y**

Suppose we have four classes:  
1. pedestrian  
2. car  
3. motorcycle  
4. background  
  
And we need to output $b_x, b_y, b_h, b_w$, class label (1 - 4)  
  
$$
\boldsymbol y = 
\begin{bmatrix}
\begin{aligned}
p_c \\
b_x \\
b_y \\
b_h \\
b_w \\
c_1 \\
c_2 \\
c_3
\end{aligned}
\end{bmatrix}
$$

$p_c$ denotes whether there is any object. And $c_1, c_2, c_3$ at most one of them can be equal to one.

**Loss function**

$$
L(\boldsymbol {\hat y}, y) = 
\left \{
\begin{aligned}
& (\boldsymbol{\hat y_1} - \boldsymbol y_1)^2 + (\boldsymbol{\hat y_2} - \boldsymbol y_2)^2 + \dots + (\boldsymbol{\hat y_8} - \boldsymbol y_8)^2 & y_1 = 1 (p_c = 1) \\
& (\boldsymbol{\hat y_1} - \boldsymbol y_1)^2 & y_1 = 0 (p_c = 0)
\end{aligned}
\right .
$$

In practice $p_c$: logistic reg loss, $b_x, b_y, b_h, b_w$: squared error, $c_1, c_2, c_3$: softmax func loss.

<font size=3>**# 3.2 Landmark detection**</font>

**Landmark Detection**

By selecting a number of landmarks, and generating a label training set that contains all of these landmarks, you can then have the nn tall you where are all the key positions or the key landmarks on a face.

$$ \boldsymbol y = 
\begin{bmatrix}
\begin{aligned}
face? \\
l_{1x} \\
l_{1y} \\
\vdots \\
l_{64x} \\
l_{64y}
\end{aligned}
\end{bmatrix}
$$

<font size=3>**# 3.3 Object detection**</font>

**Cropped imgs**

Suppose you are to build a car detection algorithm. Build a labeled training set with closely cropped examples of cars. For our purposes in this training set you can start off with closely cropped images, meaning that $\boldsymbol x$ is pretty much only the car. 

**Sliding Windows Detection**

Basically go through every region of the size of the window, and pass lots of little cropped images into the conv net and make classification. Use diff sizes of windows then so long as there's a car somewhere in the img, there will be a window can detect it. 

**Disadvantage**

Computational cost.

<font size=3>**# 3.4 Convolutional Implementation of Sliding Windows**</font>

**Turing FC Layer into Convolutional Layers**

5 * 5 * 16 -> (FC) -> a 400-unit fc layer -> (FC) -> a 400-unit fc layer -> y (softmax 4(4))  
↓  
5 * 5 * 16 -> (FC 400 5 * 5 * 16) -> 1 * 1 * 400 -> (FC 400 1 * 1 * 400) -> 1 * 1 * 400 -> (4 1 * 1 * 400) -> 1 * 1 * 4

**Convolution Implementation of Sliding Windows**

Instead of running forward prop on four subsets of the input img independently (using cropped small images), use the entire img to detect with turning fc layers into conv layers.  
  
Say that your conv net inputs 14 * 14 * 3 imgs, and your test set img is 16 * 16 * 3 (this img is generated by padding 2 rows at the bottom and 2 cols on the right).  
  
For the input 14 * 14 * 3 -> 1 * 1 * 4,
for the output 16 * 16 * 3 (stride = 2 in the test set img, max pool: 2 * 2) -> 2 * 2 * 4. (0, 0) is the output of [0:14, 0:14] in the input img in the test set, and (0, 1) the output of [2:16, 2:16], (1, 0) [2:16, 0:14], (1, 1) the output of [2:16, 2:16]. In this example [0:14, 0:14] is 1 and the others 0.

Because of the max pooling of 2 (max pool: 2 * 2) that this corresponds to running your nn with a stride of 2 on the original img. 

**Weakness**

The pos of the bouding boxes is not goling to be too accurate.

<font size=3>**# 3.5 Bounding box predictions**</font>

**YOLO Algorithm**

Let's say you have a 100 * 100 img, you're going to place down a grid on this img. A 3 * 3 grid is used here. In actual implementation you use a finer one, maybe a 19 * 19 grid. The basic idea is you're going to take the img classification and localization algorithm that you saw in the first video of this week and apply that to each of the nine grid cells of this image. 

**Labels for training**

For each grid cell:  
$$
\boldsymbol y = 
\begin{bmatrix}
\begin{aligned}
p_c \\
b_x \\
b_y \\
b_h \\
b_w \\
c_1 \\
c_2 \\
c_3
\end{aligned}
\end{bmatrix}
$$
  
9 $\boldsymbol {\hat y}$ for the img.

**What YOLO Does**

The img has 2 objs and what the YOLO algorithm does is it takes the midpoint of each of the 2 objs, and it assigns the obj to the grid cell containing the midpoint. And so even though the central grid cell has some parts of both cars, we will pretent the central grid cell has no intersting obj.  
  
The target output will be 3 * 3 * 8.  
  
Note that the algorithm runs only once for each example.

**Advantage**

Outputs precise bouding boxes. So long as you don't have more than one obj in each grid cell, this algorithm should work ok.  
  
Very fast because it uses convolution.

**Specify the Bounding Boxes**

For each grid the top left is (0, 0) and the low right is (1, 1). $b_x, b_y, b_h, b_w$ are specified **relative to** the grid cell.  
   
The height, width of the bounding box is specified as a fraction of the overall height, width of the box (grid). For example the width of the bounding box is 90% of the width of the grid, so $b_w$ is 0.9. And the height of the bounding box is 50% of the height of the grid, in that case $w_b$ is 0.5.  
  
$b_x, b_y$ must be btw 0 and 1 coz pretty much by definition the midpoint is within the bounds of that grid cell it was assigned to if it was not btw 0 and 1 if it's outside the square it would be assigined to a diff grid cell.  
  
$b_w and b_h$ can be greater than 1. For example there is an obj with a bounding box whose width and / or height are / is greater than those / that of the grid cell.

<font size=3>**# 3.6 Intersection over union**</font>

$IoU = \frac{size \ of \ intersection}{size \ of \ union}$

"Correct" if IoU $\geq$ 0.5
  
0.5 can be replaced by other values such as 0.6

<font size=3>**# 3.7 Non-max suppression**</font>

When you're implementing the detection algorithm maybe grids aroung the grid containing the midpoint also think they contain the midpoint so that the obj is detected more than once. For example there are two objs in an img more than two grid will give a vector with $p_c = 1$. Non-max suppression is a way to make sure your algorithm detects each obj only once.

**How it Works**

First the bounding box with the highest $p_c$ will be picked and the IoU between the box and other bounding boxes will be calculated. And boxes that give a high IoU will be suppressed because this means that the picked box overlaps a lot with boxes giving high IoUs. And the process is repeated.

**Non-max suppression algorithm**

Suppose we just detect cars now. Thus each output prediction is:
$$
\begin{bmatrix}
\begin{aligned}
p_c \\
b_x \\
b_y \\
b_h \\
b_w
\end{aligned}
\end{bmatrix}
$$

Discard all boxes with $p_c \leq 0.6$
While there are any remaining boxes:  
&emsp;Pick the box with the largest $p_c$, output that as a prediction  
&emsp;Discard any remaining box with $IoU \geq 0.5$ with the box output in the previous step.  
  
If multiple objs are to be detected, independently carry out non-max suppression multiple times, one on each of the outputs classes.

<font size=3>**# 3.8 Anchor Boxes**</font>

Makes a grid cell detect multiple objs.

Suppose that the midpoint of the pedestrian and the midpoint of the car are in almos the same place and both of them fall into the same grid cell. For that grid cell if y outputs the $\boldsymbol {\hat y}$ where you're detecting three classes it won't be able to output the detection.

**What Anchor Boxes Do**

For the example above predifine two diff shapes called anchor boxes with different shapes, such as a fat short one and a slim tall one. And associate two predictions with the two anchor boxes. In general you might use more anchor boxes maybe 5 or even more.  
  
If you're using two anchor boxes, then each of the nine grid cells you get two predicted bounding boxes, some of them will have very low probability very low $p_c$, but you still get two predicted bounding boxes for each of the nine grid cell. Some of the bounding boxes can go outside the height and width of the grid cell that it came from.

**The Output Vector**

Instead of the vector introduced in the previous videos, use
$$
\begin{bmatrix}
\begin{aligned}
p_c \\
b_x \\
b_y \\
b_h \\
b_w \\
c_1 \\
c_2 \\
c_3 \\
p_c \\
b_x \\
b_y \\
b_h \\
b_w \\
c_1 \\
c_2 \\
c_3
\end{aligned}
\end{bmatrix}
$$

The first eight rows are for anchor box 1 and the remaining rows for anchor box 2.

**Anchor Box Algorithm**

With two anchor boxes:  
Each objs in training image is assigned to grid cell that contains obj's midpoint and anchor box for the grid cell with highest IoU.  
  
Output now: 3 * 3 * (8 * 2)

**An Example**

Suppose anchor box 1 is a rectangle whose height is longer and anchor box whose width is longer. If there is a grid contains the midpoints of both a car and a pedestrian, the output vector will be
$$
\begin{bmatrix}
\begin{aligned}
1 \\
b_x \\
b_y \\
b_h \\
b_w \\
1 \\
0 \\
0 \\
1 \\
b_x \\
b_y \\
b_h \\
b_w \\
0 \\
1 \\
0
\end{aligned}
\end{bmatrix}
$$
  
And if the grid only contains a pedestrian:  
$$
\begin{bmatrix}
\begin{aligned}
0 \\
? \\
? \\
? \\
? \\
? \\
? \\
? \\
1 \\
b_x \\
b_y \\
b_h \\
b_w \\
0 \\
1 \\
0
\end{aligned}
\end{bmatrix}
$$

**Limitations**

It cannot handle the case that 3 objs are in the same grid cell and the case that 2 objs with the same anchor shape are in the same grid cell.

**How to Choose Anchor Boxes?**

By hand or K-means algorithm.

<font size=3>**# 3.10 Region Proposals**</font>

**Region Proposal: R-CNN (Regions with Convolutional Networks)**

It tries to pick just a few regions that make sense to run your conv net classifier on. So rather than running your sliding windows on every single window, you instead select just a few windows, and run your conv net classifier on just a few windows. The way they perform the region proposals is to run an algorithm called a segmentation algorithm. The R-CNN algorithm does not just trust the bounding box it was given, it also outputs a bounding box $b_x, b_y, b_h, b_w$ in order to get a more accurate bounding box than whatever happended to surround the block that the img segmentation algorithm gave it. 

**Downside**

Slow. Classify the regions one at a time.

**Fast R-CNN**

Faster, use conv implementation of sliding windows to classify all the proposed regions.

**Faster R-CNN**

more faser, use a conv nn to propose regions.

Improved R-CNN alrotihms are faster but still slower than YOLO. 